In [1]:
import ee
import geemap

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()


In [265]:
# Define the ROIs (already provided)
#adjusted to be purposefully sloppy around lake edges, need to include more soil 
roi_LF = ee.Geometry.Polygon([
    [163.1841307370196,-77.59677387872982],
    [163.1576948849688,-77.60097649476886],
    [163.11512286348443,-77.60665148730044],
    [163.08937365694146,-77.61166104297976],
    [163.0584746090899,-77.61777294645421],
    [163.06087786836724,-77.62189496331501],
    [163.0698042599688,-77.62174777168963],
    [163.11580950899224,-77.62285166688366],
    [163.1518583981524,-77.62071738206735],
    [163.19305712862115,-77.61733122165525],
    [163.2174330441485,-77.61180835267379],
    [163.26309497041802,-77.60738831170583],
    [163.2675581662188,-77.59913341471547],
    [163.25279528780084,-77.5969951040001],
    [163.233912536336,-77.59345503355506],
    [163.1841307370196,-77.59677387872982]
]
)

bright_roi = ee.Geometry.Polygon([
    [163.12698754679585, -77.6073690739641], 
    [163.16406640421772, -77.60946879016157], 
    [163.15445336710835, -77.61628124600912], 
    [163.11823281657124, -77.6140353843151],
    [163.12698754679585, -77.6073690739641]
])

dim_roi = ee.Geometry.Polygon([
    [162.51530564185225,-77.69439782758747],
    [162.52303040381514,-77.69677565811227],
    [162.55272782202803,-77.69223909736411],
    [162.54431641455733,-77.68997019942111],
    [162.51530564185225,-77.69439782758747]
])

# Define ROI for analysis
roi_for_cropping = ee.Geometry.Polygon([
    [163.24357885656732,-77.5812101682432],
    [162.70112890539545,-77.64219641154588],
    [162.74919409094232,-77.66275299707411],
    [162.38939184484857,-77.70376522386023],
    [162.25068945227045,-77.71808776695123],
    [162.3028745108642,-77.73968659951969],
    [163.33146948156732,-77.61630600197311],
    [163.24357885656732,-77.5812101682432]
    ]
    )

In [171]:
#MAY NOT NEED THIS IN THE SCRIPT ANYMORE
#define a new ROI that is potentially simpler
roi_box = ee.Geometry.Polygon(
    [[163.12698754679585, -77.6073690739641], 
    [163.16406640421772, -77.60946879016157], 
    [163.15445336710835, -77.61628124600912], 
    [163.11823281657124, -77.6140353843151],
    [163.12698754679585, -77.6073690739641]]
)

In [152]:
pointed = ee.Geometry.BBox(162.1, -77.73, 163.3, -77.59)

In [253]:
## add the image collection
ids = ee.List(["Sentinel-2A_2015-12-18", "Sentinel-2A_2015-12-21", "Sentinel-2A_2015-12-27",
               "Sentinel-2A_2015-12-28", "Sentinel-2A_2015-12-29", "Sentinel-2A_2015-12-31", 
               "Sentinel-2A_2016-01-07", "Sentinel-2A_2016-01-13", "Sentinel-2A_2016-01-14", 
               "Sentinel-2A_2016-01-23", "Sentinel-2A_2016-02-02", "Sentinel-2A_2016-02-05", 
               "Sentinel-2A_2016-02-19", "Sentinel-2A_2016-10-24", "Sentinel-2A_2016-10-26", 
               "Sentinel-2A_2016-10-30", "Sentinel-2A_2016-11-01", "Sentinel-2A_2016-11-03", 
               "Sentinel-2A_2016-11-05", "Sentinel-2A_2016-11-06", "Sentinel-2A_2016-11-08", 
               "Sentinel-2A_2016-11-09", "Sentinel-2A_2016-12-12", "Sentinel-2A_2016-12-13", 
               "Sentinel-2A_2016-12-15", "Sentinel-2A_2016-12-18", "Sentinel-2A_2016-12-19", 
               "Sentinel-2A_2016-12-25", "Sentinel-2A_2017-01-02", "Sentinel-2A_2017-01-08", 
               "Sentinel-2A_2017-01-14", "Sentinel-2A_2017-01-15", "Sentinel-2A_2017-01-18", 
               "Sentinel-2A_2017-01-24", "Sentinel-2A_2017-01-25", "Sentinel-2A_2017-01-27", 
               "Sentinel-2A_2017-01-28", "Sentinel-2A_2017-01-30", "Sentinel-2A_2017-01-31", 
               "Sentinel-2A_2017-02-01", "Sentinel-2A_2017-02-10", "Sentinel-2A_2017-02-11", 
               "Sentinel-2A_2017-02-14", "Sentinel-2A_2017-02-16", "Sentinel-2A_2017-02-19", 
               "Sentinel-2A_2017-02-20", "Sentinel-2A_2017-03-08", "Sentinel-2A_2018-01-12", 
               "Sentinel-2A_2018-01-23", "Sentinel-2A_2018-02-12", "Sentinel-2B_2018-02-17", 
               "Sentinel-2B_2018-10-15", "Sentinel-2B_2018-10-24", "Sentinel-2B_2018-11-24", 
               "Sentinel-2B_2019-02-21", "Sentinel-2B_2019-02-22", "Sentinel-2B_2019-09-29", 
               "Sentinel-2B_2019-10-09", "Sentinel-2B_2019-10-19", "Sentinel-2B_2019-10-20", 
               "Sentinel-2B_2019-10-29", "Sentinel-2B_2019-11-08", "Sentinel-2B_2019-11-09", 
               "Sentinel-2B_2019-11-18", "Sentinel-2B_2019-11-19", "Sentinel-2B_2020-02-06", 
               "Sentinel-2B_2020-03-07", "Sentinel-2B_2020-03-08", "Sentinel-2B_2020-10-23", 
               "Sentinel-2B_2020-10-24", "Sentinel-2B_2020-11-03", "Sentinel-2B_2020-11-12", 
               "Sentinel-2B_2020-11-22", "Sentinel-2B_2020-11-23", "Sentinel-2B_2020-12-02", 
               "Sentinel-2B_2020-12-03", "Sentinel-2B_2021-01-01", "Sentinel-2B_2021-01-12", 
               "Sentinel-2B_2021-01-13", "Sentinel-2B_2021-01-21", "Sentinel-2B_2021-01-22", 
               "Sentinel-2B_2021-01-23", "Sentinel-2B_2021-02-01", "Sentinel-2B_2021-02-11",
               "Sentinel-2B_2021-03-02", "Sentinel-2B_2021-10-08", "Sentinel-2B_2021-10-09", 
               "Sentinel-2B_2021-10-29", "Sentinel-2B_2021-11-07", "Sentinel-2B_2021-11-08", 
               "Sentinel-2B_2021-11-17", "Sentinel-2B_2021-11-18", "Sentinel-2B_2021-12-07", 
               "Sentinel-2B_2021-12-08", "Sentinel-2B_2021-12-18", "Sentinel-2B_2021-12-28", 
               "Sentinel-2B_2022-01-07", "Sentinel-2B_2022-01-16", "Sentinel-2B_2022-01-27", 
               "Sentinel-2B_2022-02-25", "Sentinel-2B_2022-10-11", "Sentinel-2B_2022-10-13",
               "Sentinel-2B_2022-10-17", "Sentinel-2B_2022-10-31", "Sentinel-2B_2022-11-02",
               "Sentinel-2B_2022-11-12", "Sentinel-2B_2022-11-16", "Sentinel-2B_2022-11-22",
               "Sentinel-2B_2022-12-06", "Sentinel-2B_2022-12-10", "Sentinel-2B_2022-12-16",
               "Sentinel-2B_2022-12-26", "Sentinel-2B_2023-01-01", "Sentinel-2B_2023-01-09",
               "Sentinel-2B_2023-01-11", "Sentinel-2B_2023-01-15", "Sentinel-2B_2023-02-03",
               "Sentinel-2B_2023-02-08", "Sentinel-2B_2023-02-13", "Sentinel-2B_2023-02-14", 
               "Sentinel-2B_2023-02-28", "Sentinel-2B_2023-03-02", "Sentinel-2B_2023-03-06", 
               "Sentinel-2B_2023-10-06", "Sentinel-2B_2023-10-08", "Sentinel-2B_2023-10-12", 
               "Sentinel-2B_2023-10-16", "Sentinel-2B_2023-11-01", "Sentinel-2B_2023-11-15", 
               "Sentinel-2B_2023-11-17", "Sentinel-2B_2023-11-21", "Sentinel-2B_2023-12-01", 
               "Sentinel-2B_2023-12-05", "Sentinel-2B_2023-12-07", "Sentinel-2B_2023-12-11", 
               "Sentinel-2B_2023-12-15", "Sentinel-2B_2024-01-04", "Sentinel-2B_2024-01-10", 
               "Sentinel-2B_2024-01-14", "Sentinel-2B_2024-01-26", "Sentinel-2B_2024-02-03", 
               "Sentinel-2B_2024-09-22", "Sentinel-2B_2024-10-06", "Sentinel-2B_2024-10-16", 
               "Sentinel-2B_2024-10-20", "Sentinel-2B_2024-10-22", "Sentinel-2B_2024-10-26", 
               "Sentinel-2B_2024-10-30", "Sentinel-2B_2024-11-09", "Sentinel-2B_2024-11-19", 
               "Sentinel-2B_2024-11-21", "Sentinel-2B_2024-11-29", "Sentinel-2B_2024-12-09", 
               "Sentinel-2B_2024-12-21", "Sentinel-2B_2024-12-23", "Sentinel-2B_2024-12-24", 
               "Sentinel-2B_2024-12-29", "Sentinel-2B_2024-12-31", "Sentinel-2B_2025-01-02", 
               "Sentinel-2B_2025-01-10", "Sentinel-2B_2025-01-18", "Sentinel-2B_2025-01-28"])

# Define start and end date for the image collection filter
start_date = "2015-03-06"
end_date = "2025-01-01"

In [258]:
def addImageDate(image):
    mission = image.get('SPACECRAFT_NAME')
    date = image.date().format('YYYY-MM-dd')
    missDate = ee.String(mission).cat('_').cat(ee.String(date))
    return image.set('missDate', missDate)

In [259]:
def mosaic_by_date(imcol):
    # Convert the image collection to a list of images
    imlist = imcol.toList(imcol.size())
    
    # Get unique dates from the image collection
    def get_date(image):
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    unique_dates = imlist.map(lambda im: get_date(im)).distinct()

    def create_mosaic(date_str):
        date = ee.Date(date_str)
        
        # Filter images for that day and create a mosaic
        mosaic = imcol.filterDate(date, date.advance(1, 'day')).mosaic()
        
        return mosaic.set({
            'system:time_start': date.millis(),
            'system:id': date.format('YYYY-MM-dd')
        })

    # Create mosaics for each unique date
    mosaic_imlist = unique_dates.map(create_mosaic)
    
    return ee.ImageCollection(mosaic_imlist)

In [260]:
# Example usage with an image collection )
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
    .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])\
    .filterDate(start_date, end_date)\
    .map(addImageDate)\
    .filter(ee.Filter.inList("missDate", ids))\
    .filter(ee.Filter.gt('MEAN_SOLAR_ZENITH_ANGLE',70))\
    .filterBounds(roi_for_cropping)\
    .sort('DATE_ACQUIRED')
s3 = mosaic_by_date(s2)

In [261]:
# Clip all the images in the s3 collection down to the ROI
def clip_image(image):
    return image.clip(roi_for_cropping)

# Apply clip to image collection
copernicus_clipped = s3.map(clip_image)
print(copernicus_clipped.size().getInfo())

61


In [262]:
# Function to calculate mean band values around a simple point
# this function will be passed through the define endmembers 
def calculate_mean_band_values(image, point_geometry):
    buffer = point_geometry.buffer(80)  #80 meter buffer radius, 4x4 pixel
    band_means = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffer,
        scale=20,
        maxPixels=1e8  # Adjusted based on sample size
    )
    return band_means

In [229]:
####### OLD VERSION OF THIS FUNCTION
# Function to define endmembers
def define_endmembers(image, roi, n=1):
   # Calculate brightness
    #brightness = image.select(['B2','B3', 'B4', 'B11', 'B12']).reduce(ee.Reducer.sum()).rename('brightness') #RGB and SWIR 1
    brightness = image.select(['B11', 'B12']).reduce(ee.Reducer.sum()).rename('brightness') #SWIR 1 and 2

    # Generate a grid of points
    points = ee.FeatureCollection.randomPoints(region=roi_LF, points=6000)

    # Calculate brightness for each point
    points_with_brightness = brightness.reduceRegions(
        collection=points,
        reducer=ee.Reducer.mean(),
        scale=20
        )

    # Sort points by brightness
    brightest_point = points_with_brightness.sort('brightness', False).first()
    brightest_geometry = ee.Feature(brightest_point).geometry()

    # Filter out points near the brightest point
    filtered_points = points_with_brightness.filter(ee.Filter.bounds(brightest_geometry.buffer(100)).Not())

    # Find the dimmest point from the remaining points
    dimmest_point = filtered_points.sort('brightness', True).first()
    dimmest_geometry = ee.Feature(dimmest_point).geometry()

    # Calculate mean band values for brightest and dimmest points
    brightest_band_means = calculate_mean_band_values(image, brightest_geometry)
    dimmest_band_means = calculate_mean_band_values(image, dimmest_geometry)

    # Combine into endmembers
    endmembers = ee.List([
     brightest_band_means.values(),
     dimmest_band_means.values()
    ])
    return endmembers


In [244]:
# NEW VERSION OF THIS FUNCTION TAKES A THREE POINT MEAN
# NEED TO UPDATE SUCH THAT THE ICE AND SOIL ENDMEMBERS ARE PULLED FROM COMPLETELY DIFFERENT ROIS
# Function to define endmembers
def define_endmembers(image, roi, n=1):
    # Calculate brightness
    brightness = copernicus_clipped.first().select(['B2', 'B3', 'B4', 'B11', 'B12']).reduce(ee.Reducer.sum()).rename('brightness')  # RGB and SWIR Bands

    # Generate a grid of points
    points = ee.FeatureCollection.randomPoints(region=roi_LF, points=2000)

    # Calculate brightness for each point
    points_with_brightness = brightness.reduceRegions(
    collection=points,
    reducer=ee.Reducer.mean(),
    scale=20
    )

    # Sort points by brightness
    sorted_points = points_with_brightness.sort('brightness', False)  # Descending order

    # Select the top 3 brightest points
    top_3_brightest = sorted_points.toList(3)
    brightest_geometry = ee.FeatureCollection(top_3_brightest).geometry()

    # Filter out points near the top 3 brightest points
    #filtered_points = points_with_brightness.filterBounds(brightest_geometry.buffer(100)).Not()
    filtered_points = points_with_brightness.filter(ee.Filter.bounds(brightest_geometry.buffer(100)).Not())

    # Sort remaining points in ascending order (dimmest first)
    sorted_filtered_points = filtered_points.sort('brightness', True)

    # Select the bottom 3 dimmest points
    bottom_3_dimmest = sorted_filtered_points.toList(3)
    dimmest_geometry = ee.FeatureCollection(bottom_3_dimmest).geometry()

    # Calculate mean band values for the brightest and dimmest points
    brightest_band_means = calculate_mean_band_values(copernicus_clipped.first(), brightest_geometry)
    dimmest_band_means = calculate_mean_band_values(copernicus_clipped.first(), dimmest_geometry)

    # Combine into endmembers
    endmembers = ee.List([
    brightest_band_means.values(),
    dimmest_band_means.values()
    ])  
    return endmembers


In [306]:
# Function to define endmembers from separate ROIs
def define_endmembers(image, bright_roi, dim_roi, n=1):
    # Calculate brightness for the bright ROI
    brightness_bright = image.select(['B2', 'B3', 'B4']).reduce(ee.Reducer.sum()).rename('brightness')

    # Generate a grid of points in the bright ROI
    bright_points = ee.FeatureCollection.randomPoints(region=bright_roi, points=2000)

    # Calculate brightness for each point in the bright ROI
    bright_points_with_brightness = brightness_bright.reduceRegions(
        collection=bright_points,
        reducer=ee.Reducer.mean(),
        scale=20
    )

    # Sort bright points by brightness (descending)
    sorted_bright_points = bright_points_with_brightness.sort('brightness', False)

    # Select the top 3 brightest points
    top_3_brightest = sorted_bright_points.toList(n)
    brightest_geometry = ee.FeatureCollection(top_3_brightest).geometry()

    # Calculate brightness for the dim ROI
    brightness_dim = image.select(['B2', 'B3', 'B4']).reduce(ee.Reducer.sum()).rename('brightness')

    # Generate a grid of points in the dim ROI
    dim_points = ee.FeatureCollection.randomPoints(region=dim_roi, points=2000)

    # Calculate brightness for each point in the dim ROI
    dim_points_with_brightness = brightness_dim.reduceRegions(
        collection=dim_points,
        reducer=ee.Reducer.mean(),
        scale=20
    )

    # Sort dim points by brightness (ascending)
    sorted_dim_points = dim_points_with_brightness.sort('brightness', True)

    # Select the bottom 3 dimmest points
    bottom_3_dimmest = sorted_dim_points.toList(n)
    dimmest_geometry = ee.FeatureCollection(bottom_3_dimmest).geometry()

    # Calculate mean band values for the brightest and dimmest points
    brightest_band_means = calculate_mean_band_values(image, brightest_geometry)
    dimmest_band_means = calculate_mean_band_values(image, dimmest_geometry)

    # Combine into endmembers
    endmembers = ee.List([
        brightest_band_means.values(),
        dimmest_band_means.values()
    ])  

    return endmembers


In [307]:
outputs = define_endmembers(copernicus_clipped.first(), bright_roi, dim_roi)
print(outputs.getInfo())

[[285.2080198806484, 279.52469021013303, 4222.123848325529, 3886.79561476172, 4166.927560732336, 3307.406099437052], [2028.669354567751, 1939.6299768168524, 2245.7215334475686, 1949.8228841178643, 2021.3082686557127, 1941.684423613211]]


In [308]:
def unmix_image_collection(image_collection, bright_roi, dim_roi):
    def unmix_image(image):
        # Define endmembers for the current image
        endmembers = define_endmembers(image, bright_roi, dim_roi)

        # Apply the unmixing process
        unmixed = image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']).unmix(endmembers, True, True)

        # Rename unmixed bands for clarity
        unmixed = unmixed.rename(['endmember_1', 'endmember_2'])

        # Add properties for tracking
        return unmixed.set('system:time_start', image.get('system:time_start'))

    # Map the unmixing process over the image collection
    unmixed_collection = image_collection.map(unmix_image)

    return unmixed_collection

In [309]:
# apply process to all images in collection
result_collection = unmix_image_collection(copernicus_clipped, bright_roi, dim_roi)
print(result_collection.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'endmember_1', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [3, 2], 'origin': [161, -79], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'endmember_2', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [3, 2], 'origin': [161, -79], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': 1455840000000, 'system:index': '0'}}


In [298]:
image = copernicus_clipped.first()
n = 3

# Calculate brightness for the bright ROI
brightness_bright = image.select(['B2', 'B3', 'B4']).reduce(ee.Reducer.sum()).rename('brightness')

    # Generate a grid of points in the bright ROI
bright_points = ee.FeatureCollection.randomPoints(region=bright_roi, points=2000)

    # Calculate brightness for each point in the bright ROI
bright_points_with_brightness = brightness_bright.reduceRegions(
        collection=bright_points,
        reducer=ee.Reducer.mean(),
        scale=20
    )

    # Sort bright points by brightness (descending)
sorted_bright_points = bright_points_with_brightness.sort('brightness', False)

    # Select the top 3 brightest points
top_3_brightest = sorted_bright_points.toList(n)
brightest_geometry = ee.FeatureCollection(top_3_brightest).geometry()

    # Calculate brightness for the dim ROI
brightness_dim = image.select(['B2', 'B3', 'B4']).reduce(ee.Reducer.sum()).rename('brightness')

    # Generate a grid of points in the dim ROI
dim_points = ee.FeatureCollection.randomPoints(region=dim_roi, points=2000)

    # Calculate brightness for each point in the dim ROI
dim_points_with_brightness = brightness_dim.reduceRegions(
        collection=dim_points,
        reducer=ee.Reducer.mean(),
        scale=20
    )

    # Sort dim points by brightness (ascending)
sorted_dim_points = dim_points_with_brightness.sort('brightness', True)

    # Select the bottom 3 dimmest points
bottom_3_dimmest = sorted_dim_points.toList(n)
dimmest_geometry = ee.FeatureCollection(bottom_3_dimmest).geometry()

    # Calculate mean band values for the brightest and dimmest points
brightest_band_means = calculate_mean_band_values(image, brightest_geometry)
dimmest_band_means = calculate_mean_band_values(image, dimmest_geometry)

    # Combine into endmembers
endmembers = ee.List([
        brightest_band_means.values(),
        dimmest_band_means.values()
    ])  

In [310]:
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # Red, Green, Blue bands (for Sentinel-2 or Landsat 8)
    'min': 0,
    'max': 10000,  # Adjust based on the dataset's reflectance range
    'gamma': 1.4  # Optional, enhances contrast
}

brightness_params = {
    "min": 0,
    "max": 10000,  # Adjust based on expected brightness values
    "palette": ["black", "blue", "green", "yellow", "red"],
}

#Plot to confirm output
Map = geemap.Map(zoom=10, center=[-77.616808, 163.077952])
Map.centerObject(roi_LF, 10)
Map.addLayer(result_collection.first())
Map.addLayer(copernicus_clipped.first(), vis_params)
Map.addLayer(brightness_bright, brightness_params)
Map.addLayer(dimmest_geometry)
Map.addLayer(brightest_geometry)
Map.addLayer(dim_roi)
Map.addLayer(bright_roi)

Map

Map(center=[-77.6091387013167, 163.16942980628684], controls=(WidgetControl(options=['position', 'transparent_…

In [313]:
# export the images in the copernicus_clipped collection
def export_image(image):
    date_str = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'Sentinel2_unmixed_images_{date_str}',
        folder='EarthEngine',  # Replace with your folder name
        scale=20,                 # Define the scale (e.g., 30 meters per pixel)
        crs='EPSG:3031',          # Define the Coordinate Reference System
        region=pointed,  # Define the region to export
        fileNamePrefix=f'Sentinel_20deg_unmix_feb18_{date_str}'
    )
    task.start()
    print(f"Export started for image with date {date_str}.")
    

In [314]:
def process_images():
    image_list = result_collection.toList(result_collection.size().getInfo())
    num_images = image_list.size().getInfo()
    
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        export_image(image)

process_images()

EEException: Too many concurrent aggregations.